In [7]:
from collections import defaultdict

#f = open("dataset_865451_8.txt","r")
f = open("sample2.txt","r")

match_reward, mismatch_penalty, gap_open_penalty, gap_extension_penalty = [float(x) for x in next(f).split()]
v, w = f.readline().strip('\n'), f.readline().strip('\n')
f.close()

def AffineAlignment(match_reward, mismatch_penalty, gap_open_penalty, gap_extension_penalty, v, w):
    n = len(v) + 1
    m = len(w) + 1
    Backtrack = [[" "]*m for i in range(n)]
    DAG = defaultdict(lambda: (-1,-1,-1)) #DAG: (tier, i, j), tiers: lower = 0, middle = 1, upper = 2

    
    #setup three sets of nodes
    middle = [[0]*(m) for i in range(n)]
    middle[0][1], middle[1][0] = -gap_open_penalty, -gap_open_penalty
    for i in range(2,n):
        middle[i][0] = -gap_extension_penalty * (i - 1) - gap_open_penalty
    for j in range(2,m):
        middle[0][j] = -gap_extension_penalty * (j - 1) - gap_open_penalty
        
    lower = [[float('-inf')]*(m) for i in range(n)]
    upper = [[float('-inf')]*(m) for i in range(n)]
    
    #fill out the three matrices using dynamic programming
    for i in range(1, n):
        for j in range(1, m):
            match = -(mismatch_penalty)
            if v[i - 1] == w[j - 1] or w[j - 1] == 'N':
                match = match_reward
            
            lower[i][j] = max(lower[i-1][j] - gap_extension_penalty, middle[i-1][j] - gap_open_penalty)
            upper[i][j] = max(upper[i][j-1] - gap_extension_penalty, middle[i][j-1] - gap_open_penalty)
            middle[i][j] = max(lower[i][j], upper[i][j], middle[i-1][j-1] + match)
            
            if lower[i][j] == lower[i-1][j] - gap_extension_penalty:
                DAG[(0,i,j)] = (0,i-1,j)
            else:
                DAG[(0,i,j)] = (1,i-1,j) 
                
            if upper[i][j] == upper[i][j-1] - gap_extension_penalty:
                DAG[(2,i,j)] = (2,i,j-1)
            else:
                DAG[(2,i,j)] = (1,i,j-1)
                
            if middle[i][j] == lower[i][j]:
                DAG[(1,i,j)] = (0,i,j)
            elif middle[i][j] == upper[i][j]:
                DAG[(1,i,j)] = (2,i,j)
            else:
                DAG[(1,i,j)] = (1,i-1,j-1)
    
    
    #setup for backtracking
    aligned1 = ""
    aligned2 = ""
    curr_node = (-1,-1,-1)
    max_length = 0
    
    
    #determine which of the three matrices has the maximum path
    if lower[n-1][m-1] >= middle[n-1][m-1] and lower[n-1][m-1] >= upper[n-1][m-1]:
        max_length = lower[n-1][m-1]
        curr_node = (0,n-1,m-1)
    elif middle[n-1][m-1] >= lower[n-1][m-1] and middle[n-1][m-1] >= upper[n-1][m-1]:
        max_length = middle[n-1][m-1]
        curr_node = (1,n-1,m-1)
    else:
        max_length = upper[n-1][m-1]
        curr_node = (2,n-1,m-1)
    
    
    #backtrack using the DAG
    i,j = n-1,m-1
    while DAG[curr_node] != (-1,-1,-1):
        temp_node = DAG[curr_node]
        
        if temp_node[0] == 0 and curr_node[0] == 0:
            aligned1 = v[i-1] + aligned1
            aligned2 = "-" + aligned2
            i -= 1
        elif temp_node[0] == 1 and curr_node[0] == 0:
            aligned1 = v[i-1] + aligned1
            aligned2 = "-" + aligned2
            i -= 1
            
        elif temp_node[0] == 1 and curr_node[0] == 1:
            aligned1 = v[i-1] + aligned1
            aligned2 = w[j-1] + aligned2
            if (v[i-1] != w[j-1]) and i-1 > 150 and i-1 < 300:
                print(v[i-1], "->", w[j-1])
            i -= 1
            j -= 1
            
        elif temp_node[0] == 2 and curr_node[0] == 2:
            aligned1 = "-" + aligned1
            aligned2 = w[j-1] + aligned2
            j -= 1
        elif temp_node[0] == 1 and curr_node[0] == 2:
            aligned1 = "-" + aligned1
            aligned2 = w[j-1] + aligned2
            j -= 1
            
        curr_node = temp_node
    
    
    #fill in leading dashes
    while i != 0:
        aligned1 = v[i-1] + aligned1
        aligned2 = "-" + aligned2
        i -= 1
    while j != 0: 
        aligned1 = "-" + aligned1
        aligned2 = w[j-1] + aligned2
        j -= 1
    
    #return
    return max_length, aligned1, aligned2


for item in AffineAlignment(match_reward, mismatch_penalty, gap_open_penalty, gap_extension_penalty, v, w):
    print(item)

A -> G
A -> T
G -> A
G -> T
G -> A
T -> C
A -> T
A -> G
650.3999999999999
TTGTTGCATGGCTACGAACCGGGTAATACTAAGTGATTGACTCTTGCTGACCTTTTATTAAGAACTAAATGGACAATATTATGGAGCATTTCATGTATAAATTGGTGCGTAAAATCGTTGGATCTCTCTTCTAAGTACATCCTACTATAACAATCAAGAAAAACAAGAAAATCGGACAAAACAATCAAGTATGGATTCTAGAACAGTTGGTATATTAGGAGGGGGACAATTGGGACGTATGATTGTTGAGGCAGCAAACAGGCTCAACATTAAGACGGTAATACTAGATGCTGAAAATTCTCCTGCCAAACAAATAAGCAACTCCAATGACCACGTTAATGGCTCCTTTTCCAATCCTCTTGATATCGAAAAACTAGCTGAAAAATGTGATGTGCTAACGATTGAGATTGAGCATGTTGATGTTCCTACACTAAAGAATCTTCAAGTAAAACATCCCAAATTAAAAATTTACCCTTCTCCAGAAACAATCAGATTGATACAAGACAAATATATTCAAAAAGAGCATTTAATCAAAAATGGTATAGCAGTTACCCAAAGTGTTCCTGTGGAACAAGCCAGTGAGACGTCCCTATTGAATGTTGGAAGAGATTTGGGTTTTCCATTCGTCTTGAAGTCGAGGACTTTGGCATACGATGGAAGAGGTAACTTCGTTGTAAAGAATAAGGAAATGATTCCGGAAGCTTTGGAAGTACTGAAGGATCGTCCTTTGTACGCCGAAAAATGGGCACCATTTACTAAAGAATTAGCAGTCATGATTGTGAGATCTGTTAACGGTTTAGTGTTTTCTTACCCAATTGTAGAGACTATCCACAAGGACA
NNNN-------------------------------ATTGACTCTTGCTGACCTTTTATTAAGAACTAAATGGACAATATTATGGAG